## Imports

In [20]:
## main functions
import gene_symptoms_question_functions as gsf
## importlib makes sure functions from packages get refreshed
import importlib
import requests
importlib.reload(gsf)
import pandas as pd
import math
from biothings_explorer.user_query_dispatcher import FindConnection
from biothings_explorer.hint import Hint
ht = Hint()

## Set disease and output names

In [21]:
disease_name = 'Factor V Leiden mutation'
output_csv = "Factor_V_Leiden_2020-08-27.csv"
symptom_csv = "Factor_V_Leiden_Symptoms_2020-08-27.csv"

## Get disease 

In [22]:
disease_name = disease_name.lower()
disease = ht.query(disease_name)['Disease'][0]
disease

{'UMLS': 'C0584960',
 'name': 'Factor V Leiden mutation',
 'primary': {'identifier': 'UMLS', 'cls': 'Disease', 'value': 'C0584960'},
 'display': 'UMLS(C0584960) name(Factor V Leiden mutation)',
 'type': 'Disease'}

## Get disease symptoms

In [23]:
# get disease symptoms
disease_symptoms_list = gsf.get_disease_symptoms('Thrombophilia')
disease_symptoms = disease_symptoms_list[0]
disease_symptom_hpids = disease_symptoms_list[1]
disease_symptom_dict = disease_symptoms_list[2]
disease_symptoms
disease_symptom_hpids

['recurrent thromboembolism', 'hypercoagulability', 'Blood hyperviscosity', 'Thrombophilia', 'abnormal thrombosis', 'Abnormal blood clot', 'Abnormal blood clotting', 'autosomal dominant inheritance', 'Autosomal dominant']


['HP:0004831', 'HP:0100724', 'HP:0001977', 'HP:0000006']

## Get Symptom Prevalence - based off edges out counts

In [10]:
disease_symptom_dict = gsf.get_symtpom_prevalence(disease_symptom_dict, disease_name) 
disease_symptoms_df = pd.DataFrame.from_dict(disease_symptom_dict, orient='index').sort_values(by=['edges_out_count'])
disease_symptoms_df = pd.concat([disease_symptoms_df[disease_symptoms_df["frequency"] == "Very frequent"],
               disease_symptoms_df[disease_symptoms_df["frequency"] == "Frequent"],
               disease_symptoms_df[disease_symptoms_df["frequency"] == "Occasional"],
               disease_symptoms_df[disease_symptoms_df["frequency"] == "Rare"],
               disease_symptoms_df[disease_symptoms_df["frequency"] == "Unknown"]
              ])


NameError: name 'disease_symptom_dict' is not defined

## Calculate Individual Symptom Scores

In [ ]:
individual_symptom_scores = [];
for index,x in disease_symptoms_df.iterrows():
    if(x["frequency"] == "Very frequent"):
        score = 20 / math.sqrt(int(x["edges_out_count"]))
    elif(x["frequency"] == "Frequent"):
        score = 15 / math.sqrt(int(x["edges_out_count"]))
    elif(x["frequency"] == "Occasional"):
        score = 10 / math.sqrt(int(x["edges_out_count"]))
    elif(x["frequency"] == "Rare"):
        score = 5 / math.sqrt(int(x["edges_out_count"]))
    elif(x["frequency"] == "Unknown"):
        score = 5 / math.sqrt(int(x["edges_out_count"]))
    individual_symptom_scores.append(score)

# print(len(individual_symptom_scores))
disease_symptoms_df["ISS"] = individual_symptom_scores

## Symptoms Table

In [ ]:
disease_symptoms_df

In [ ]:
disease_symptoms_df.to_csv(symptom_csv, index = False)

## Get Disease -> Gene (no intermediates) 

In [ ]:
disease_disease_to_genes_dict = gsf.get_disease_to_gene_results(disease)
disease_disease_to_genes_dict

## Get Disease -> intermediate_node (any except those matching symptoms) -> Gene

In [ ]:
# get results using intermediates 
node_type_list = (['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
                'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
                'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'])

In [ ]:
disease_disease_all_nodes_genes = gsf.predict_many(disease, node_type_list, 'Gene')

# Save an object to a file
%store disease_disease_all_nodes_genes


In [ ]:
# Restore the object
%store -r disease_disease_all_nodes_genes
disease_disease_all_nodes_genes

In [ ]:
## Remove UMLS "Genes"  (not gene specific) 
disease_disease_all_nodes_genes = disease_disease_all_nodes_genes[~disease_disease_all_nodes_genes['output_id'].str.contains('UMLS')]

In [ ]:
ht.query("nudix hydrolases")
## C1705526 "TP53 wt Allele"
# # ht.query("C0916840")
# ht.query("C1705526")

In [ ]:
ht.query("Histone Deacetylase")
ht.query("EP300 wt Allele")
ht.query("nudix hydrolases")

In [ ]:
max_2_step_genes = 100
disease_disease_to_node_to_genes_dict = gsf.get_disease_to_node_to_gene_results(disease_disease_all_nodes_genes, max_2_step_genes, disease_symptoms, disease_symptom_hpids)
%store disease_disease_to_node_to_genes_dict

## Combine genes from one and two step processes then get rid of any duplicates

In [ ]:
%store -r disease_disease_to_node_to_genes_dict
disease_top_genes_list = disease_disease_to_genes_dict["disease_to_genes_list"] + disease_disease_to_node_to_genes_dict["top_related_genes_to_disease"]
disease_top_genes_list = list(dict.fromkeys(disease_top_genes_list))
# top_genes_list

## Get Genes -> Symtoms, filtered by those related to Disease

In [ ]:
disease_relevant_genes_to_symptoms_df = gsf.determined_genes_to_symptoms(disease_top_genes_list, disease_symptoms)
# relevant_genes_to_symptoms_df

In [ ]:
%store disease_relevant_genes_to_symptoms_df

In [ ]:
%store -r disease_relevant_genes_to_symptoms_df
disease_relevant_genes_list = list(dict.fromkeys(list(disease_relevant_genes_to_symptoms_df["input"])))
# relevant_genes_list

In [ ]:
disease_gene_to_symptom_pub_counts = gsf.get_gene_to_symptom_publication_counts(disease_relevant_genes_to_symptoms_df)
# gene_to_symptom_pub_counts

In [ ]:
disease_causes_dict = gsf.create_causes_dict(disease_relevant_genes_to_symptoms_df)
# causes_dict

## Get edges out from each gene
For use in normalizing in relevance score

In [ ]:
disease_connection_dict =  gsf.get_connection_normalizing_count(disease_relevant_genes_list,node_type_list)
# connection_dict

In [ ]:
%store disease_connection_dict

In [ ]:
%store -r disease_connection_dict

## Assemble final results

In [ ]:
disease_df = gsf.assemble_final_data_frame(disease_relevant_genes_to_symptoms_df, 
                                      disease_connection_dict,
                                      disease_disease_to_genes_dict['sorted_disease_to_genes'], 
                                      disease_disease_to_node_to_genes_dict['sorted_disease_to_all_nodes_to_genes'], 
                                      disease_disease_to_node_to_genes_dict['top_two_step_genes_pub_counts'], 
                                      disease_gene_to_symptom_pub_counts, 
                                      disease_causes_dict,
                                      disease_symptoms_df)
disease_df

## Save Results

In [ ]:
disease_df.to_csv(output_csv, index = False)